<a href="https://colab.research.google.com/github/jproctor-rebecca/DS/blob/main/module3-topic-modeling/LS_DS_413_Document_Classification_Assignment_RJProctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

[David Batista - Document Classification](http://www.davidsbatista.net/blog/2017/04/01/document_classification/)

[SKLearn Pipeline - chaining estimators](https://scikit-learn.org/0.15/modules/pipeline.html#:~:text=Pipeline%20can%20be%20used%20to,feature%20selection%2C%20normalization%20and%20classification.&text=The%20last%20estimator%20may%20be,%2C%20classifier%2C%20etc)



In [ ]:
# Dependencies for the week (instead of conda)
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/requirements.txt
!pip install -r requirements.txt

--2020-10-28 16:44:48--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     137  --.-KB/s    in 0s      

2020-10-28 16:44:49 (6.31 MB/s) - ‘requirements.txt’ saved [137/137]

     |████████████████████████████████| 24.2MB 1.5MB/s 
     |████████████████████████████████| 1.6MB 43.2MB/s 
     |████████████████████████████████| 10.4MB 11.5MB/s 
     |████████████████████████████████| 7.1MB 41.1MB/s 
     |████████████████████████████████| 215kB 37.2MB/s 
     |████████████████████████████████| 2.2MB 42.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.w

In [ ]:
!python -m spacy download en_core_web_lg  # Can do lg, takes awhile
# Also on Colab, need to restart runtime after this step!

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=a87151325bca435f4e9b5fdcb7c763e14211691c5cc7708db1ec814728a6d16f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ownw2_ls/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [126]:
# Import Statements
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg

from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

from scipy.stats import randint


In [ ]:
# dataset
# target
categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
# train/test split

# load training set
train = pd.read_csv('./train.csv')

# split training set
X_train = np.array(train['description'])
y_train = np.array(train['ratingCategory'])

# load testing set
test = pd.read_csv('./test.csv')

# split testing set
X_test = np.array(test['description'])
y_test = np.array(test['ratingCategory'])

In [ ]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


### Define Pipeline Components

In [117]:
# Create Pipeline Components
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
#gbc = GradientBoostClassifier()
#gpc = GaussianProcessClassifier()

pipe = Pipeline([
                 ('vect', vect), 
                 ('clf', rfc)
                 ])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
# class sklearn.ensemble.RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, 
                                                # min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                                                # max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                                                # bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                                                # warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

# class sklearn.ensemble.GradientBoostingClassifier(*, loss='deviance', learning_rate=0.1, n_estimators=100, 
                                                    # subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
                                                    # min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, 
                                                    # min_impurity_decrease=0.0, min_impurity_split=None, init=None, 
                                                    # random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, 
                                                    # warm_start=False, presort='deprecated', validation_fraction=0.1, 
                                                    # n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

# class sklearn.gaussian_process.GaussianProcessClassifier(kernel=None, *, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=0, 
                                                          # max_iter_predict=100, warm_start=False, copy_X_train=True, 
                                                          # random_state=None, multi_class='one_vs_rest', n_jobs=None)


In [ ]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500, 1000),
    'clf__max_features': (500, 1000),
    'clf__max_depth': (15, 20),
    #'clf_criterion:' ('gini', 'entropy'),
    'clf__n_estimators=100':(100, 200),
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

In [76]:
grid_search.best_score_

0.9288181694546443

In [ ]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [77]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [78]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [79]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [ ]:
#subNumber = 0

In [80]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'submission{subNumber}_RJP.csv', index=False)


## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [82]:
# 
svd = TruncatedSVD(n_components=200, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [84]:
# LSI
lsi = Pipeline([
                ('vect', vect), 
                ('svd', svd)
                ])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', gpc)])


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [91]:
parameters = {
    'lsi__svd__n_components': [10, 100, 250],
    'lsi__vect__max_df': (0.75, 1.0),
    #'clf__max_depth':(5, 10, 15, 20),
    #'clf__max_features': (500, 1000),
    #'clf__n_estimators':(100, 200),
}

print(pipe)


Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

In [92]:
# Fit
grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 240 out of 240 | elapsed: 70.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [93]:
grid_search.best_score_

0.9206378348973209

### Make a Submission File

In [94]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [95]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [96]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [97]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model


# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'submission{subNumber}_RJP.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [98]:
# Apply to your Dataset

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

In [99]:
# Continue Word Embedding Work Here
nlp = en_core_web_lg.load()

doc = nlp('Two bananas in pyjamas')

bananas_vector = doc.vector
print(len(bananas_vector))

300


In [100]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [101]:
X = get_word_vectors(train['description'])

len(X) == len(data.data)

False

In [102]:
X_test = get_word_vectors(test['description'])

In [103]:
gpc.fit(X, train['ratingCategory'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [104]:
gpc.score(X, train['ratingCategory'])

0.8852459016393442

In [105]:
gpc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0])

In [106]:
test['ratingCategory'] = gpc.predict(X_test)

In [107]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission3_RJP.csv', header=True, index=False)

In [108]:
grid_search.best_score_

0.9206378348973209

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [ ]:
# Save your Submission File

# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
subNumber += 1
submission.to_csv(f'submission{subNumber}_RJP.csv', index=False)

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?